In [1]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '1.1.2'


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
        r = fromCopyString(variant[3])
    return A, B, L, r

In [2]:
A, B, L, r = получить_матрицы_варианта(8)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)
r = sympy.Matrix(r)

In [3]:
def матрица_b(B, r):
    return B*r

In [4]:
b = матрица_b(B, r)
b

Matrix([
[9],
[5],
[1]])

In [7]:
def матрица_управляемости(A, B, L):
    матрица = sympy.zeros(3, 6)
    матрица[:, 0:2] = B 
    матрица[:, 2:4] = A*B
    матрица[:, 4:] = A**2*B
    return матрица

In [16]:
S = матрица_управляемости(A, B, L)
S

Matrix([
[ 5, 1, 14, 13, 66, 93],
[-3, 2, -2,  7, 16, 34],
[ 1, 0,  4,  0, 16,  0]])

In [17]:
def определитель_матрицы_управляемости(S):
    определитель = S[:, :3].det()
    # print('Да!' if определитель != 0 else 'Нет :(')
    return определитель

In [18]:
print(определитель_матрицы_управляемости(S))

22


In [19]:
def коэффициенты_характеристического_полином_разомкнутой_системы(A):
    l = sympy.symbols('lambda')
    I = sympy.eye(3)
    detChPolyOpen = (l*I - A).det()
    c0 = detChPolyOpen.coeff(l, 0)
    c1 = detChPolyOpen.coeff(l, 1)
    c2 = detChPolyOpen.coeff(l, 2)
    ChPolyOpen = sympy.Matrix([
        c0,
        c1,
        c2
        ])
    return ChPolyOpen

In [20]:
коэффициенты_характеристического_полином_разомкнутой_системы(A)

Matrix([
[-44],
[ 43],
[-12]])

In [21]:
def коэффициенты_характеристического_полином_замкнутой_системы(L):
    c0 = L[0]*L[1]*L[2]
    c1 = L[0]*L[1] + L[1]*L[2] + L[0]*L[2]
    c2 = L[0]+L[1]+L[2]

    ChPolyOpen = sympy.Matrix([
        -c0,
        c1,
        -c2
        ])
    return ChPolyOpen

In [22]:
коэффициенты_характеристического_полином_замкнутой_системы(L)

Matrix([
[4],
[8],
[5]])

In [23]:
def матрица_D_обратная(A, B, L, r):
    c0, c1, c2 = коэффициенты_характеристического_полином_разомкнутой_системы(A)
    k0, k1, k2 = коэффициенты_характеристического_полином_замкнутой_системы(L)
    d3 = B*r
    d2 = A*d3 + c2*(B*r)
    d1 = A*d2 + c1*(B*r)
    res = sympy.eye(3)
    res[:, 0] = d1
    res[:, 1] = d2
    res[:, 2] = d3
    return res.inv().n()

In [25]:
toCopyString(матрица_D_обратная(A, B, L, r))

'matrix([-0.0151515151515152,0.0757575757575758,-0.242424242424242],[0,0.166666666666667,-0.833333333333333],[0.166666666666667,0.500000000000000,-3.00000000000000])'

In [27]:
def вектор_G(A, L):
    return (коэффициенты_характеристического_полином_разомкнутой_системы(A) -\
        коэффициенты_характеристического_полином_замкнутой_системы(L)).T

In [31]:
def вектор_управления(A, B, L, r):
    G = вектор_G(A, L)
    D_обр = матрица_D_обратная(A, B, L, r)
    return G*D_обр

In [33]:
toCopyString(вектор_управления(A, B, L, r))

'matrix([-2.10606060606061,-6.30303030303030,33.4696969696970])'

In [38]:
def векторное_управление_u(A, B, L, r):
    u = вектор_управления(A, B, L, r)    
    return r*u

In [39]:
res = векторное_управление_u(A, B, L, r)
res

Matrix([
[-2.10606060606061,  -6.3030303030303,  33.469696969697],
[-8.42424242424242, -25.2121212121212, 133.878787878788]])

In [40]:
toCopyString(res)

'matrix([-2.10606060606061,-6.30303030303030,33.4696969696970],[-8.42424242424242,-25.2121212121212,133.878787878788])'

In [24]:
for var_number in range(1, 31):
    A, B, L, r = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)
    r = sympy.Matrix(r)
    
    S                        = матрица_управляемости(A, B, L)
    определитель             = определитель_матрицы_управляемости(S)
    коэффициенты_разомкнутой = коэффициенты_характеристического_полином_разомкнутой_системы(A)
    коэффициенты_замкнутой   = коэффициенты_характеристического_полином_замкнутой_системы(L)
    D_обратная               = матрица_D_обратная(A, B, L, r)
    вектор_v                 = вектор_управления(A, B, L, r)
    векторное_u              = векторное_управление_u(A, B, L, r)
    
    with open(f'./results/{var_number}.txt', 'w') as file:
        file.write(toCopyString(S) + '\n')
        file.write(str(определитель) + '\n')
        file.write(toCopyString(коэффициенты_разомкнутой) + '\n')
        file.write(toCopyString(коэффициенты_замкнутой) + '\n')
        file.write(toCopyString(D_обратная) + '\n')
        file.write(toCopyString(вектор_v) + '\n')
        file.write(toCopyString(векторное_u) + '\n')
        